## Data splitting
Again, this itterates on the dataset creation process. It takes the full dataset and splits it into the datasets described in the notes of the paper. First we will be splitting the data on a per domain basis. The for each domain, we will be extracting, for each label, a separate dataset with only the pages on the domain containing that label.

In [ ]:
%matplotlib inline
# standard library
import itertools
import sys, os
import re

from urllib.parse import urlparse

# pandas
import pandas as pd
import dask.dataframe as dd

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# skelearn
from sklearn.preprocessing import LabelBinarizer

# local imports
sys.path.append(os.path.join(os.getcwd(), "src"))
from utils import get_domain_from_url

# this styling is purely my preference
# less chartjunk
sns.set_context('notebook', font_scale=1.5, rc={'line.linewidth': 2.5})
sns.set(style='ticks', palette='Set2')

In [ ]:
# read it
data = dd.read_csv('../data/ecommerce-new/final/dom-full-*.csv').drop('Unnamed: 0', axis='columns')

In [ ]:
# get url stats
sum_df = data.loc[:, ['url', 'detail_description_label', 'detail_image_label', 'detail_price_label',
       'detail_title_label', 'list_image_label', 'list_price_label',
       'list_title_label']]
sum_df = sum_df.groupby('url').sum().reset_index()
sum_df.assign(domain=sum_df.loc[:, 'url'].apply(lambda x: get_domain_from_url(x), meta=('domain', str))).groupby('domain').sum().compute()

In [ ]:
# compute how many pages per site with each label
per_page = (sum_df.set_index('url').loc[:, ['detail_description_label', 'detail_image_label', 'detail_price_label',
       'detail_title_label', 'list_image_label', 'list_price_label', 'list_title_label']] > 0).reset_index()

per_page = per_page.assign(domain=per_page.loc[:, 'url'].apply(lambda x: get_domain_from_url(x), meta=('domain', str)))

In [ ]:
# get the domains
domains = per_page['domain'].unique()
domains.compute()

In [ ]:
per_page.groupby('domain').sum().compute()

### Selection
For each site, we will be selecting, as follows:
* the pages of the entire website(aka. 5 pages containing each type of label - may overlap, and 5 containing no label whatsoever)
* 5 pages containing each type of label

In [ ]:
label_names = ['detail_description_label', 'detail_image_label', 'detail_price_label', 'detail_title_label', 'list_image_label', 'list_price_label', 'list_title_label']
domains = per_page['domain'].unique().compute()

per_page = per_page.compute()

In [ ]:
# filter by urls and labels
for label_name in label_names:
    for domain in domains:
        # get the urls fitting the descriptions
        mask_with_label = per_page[label_name] & (per_page['domain'] == domain)
        
        # get 10 pages each
        urls_with_label = per_page.loc[mask_with_label, 'url'].iloc[:10]
        filename = '{0}-{1}.csv'.format(domain, label_name)
        
        # write them to csvs
        data.loc[data['url'].isin(urls_with_label), :].compute().to_csv('../data/ecommerce-new/final/split-label/' + filename)


In [9]:
# fitler and split by urls
# for each domain get 10 urls for each label and 10 without any
for domain in domains:
    urls = set()  # urls to use
    # get 10 urls for each label
    for label_name in label_names:
        # get the urls fitting the descriptions
        mask_with_label = per_page[label_name] & (per_page['domain'] == domain)
        urls_with_label = per_page.loc[mask_with_label, 'url'].iloc[:10]
        
        urls |= set(urls_with_label) # append them
    
    # get another 10 without any label on the domain
    mask_without_label = (~per_page[label_names].any(axis='columns')) &  (per_page['domain'] == domain)
    urls_without_label = per_page.loc[mask_without_label, 'url'].iloc[:10]
    urls |= set(urls_without_label)

    # print for debug
    print(domain + '\n' + '='*80)
    print(urls)
    
    # write to csv, select only those in urls
    filename = '{0}.csv'.format(domain)
    data.loc[data['url'].isin(urls), :].compute().to_csv('../data/ecommerce-new/final/split-url/' + filename)

www.piata-az.ro
{'http://www.piata-az.ro/anunturi/agricole-24', 'http://www.piata-az.ro/anunturi/articole-copii-2124', 'http://www.piata-az.ro/anunturi/diverse-27', 'http://www.piata-az.ro/anunturi/carti-reviste-18', 'http://www.piata-az.ro/anunturi/calculatoare-16', 'http://www.piata-az.ro/', 'http://www.piata-az.ro/anunturi/apartamente-de-vanzare-1001?page=2', 'http://www.piata-az.ro/anunturi/auto-moto-velo-10', 'http://www.piata-az.ro/anunturi/animale-de-companie-23', 'http://www.piata-az.ro/anunturi/afaceri-20'}
lajumate.ro
{'https://lajumate.ro/cablu-curent-si-tablou-cu-o-siguranta-3220494.html', 'https://lajumate.ro/anunturi_afaceri-servicii.html', 'https://lajumate.ro/brichete-rumegus-fag-stejar-asigur-transport-2562379.html', 'https://lajumate.ro/anunturi_casa-gradina_3.html', 'https://lajumate.ro/angajam-confectionari-tamplarie-pvc-si-aluminiu-5490631.html', 'https://lajumate.ro/tag_Audi+a4.html', 'https://lajumate.ro/', 'https://lajumate.ro/angajam-operatoare-jocuri-2453699.h

www.emag.ro
{'https://www.emag.ro/placa-video-asus-geforcer-gtx-1080-strix-advanced-8gb-gddr5x-256-bit-strix-gtx1080-a8g-gaming/pd/DF1YK2BBM/#used-products', 'https://www.emag.ro/placa-video-msi-11gb-gddrx5-392-bit-gtx-1080-ti-gaming-x-11g/pd/DS1GMDBBM/#used-products', 'https://www.emag.ro/hota-incorporabila-telescopica-whirlpool-putere-de-absorbtie-185-mc-h-1-motor-60-cm-inox-akr-641-gy/pd/E6TXDBBBM/', 'https://www.emag.ro/all-departments?ref=hdr_mm_14', 'https://www.emag.ro/homepage', 'https://www.emag.ro/resigilate/placi_video/brand/asus/c', 'https://www.emag.ro/fashion/d', 'https://www.emag.ro/compare//', 'https://www.emag.ro/campaign/reduceri-de-stoc?ref=hdr_reduceri-de-stoc', 'https://www.emag.ro/ventilator-noctua-nf-p14s-redux-900-140x140x25-mm-ventp14s900rdx/pd/DH581BBBM/#resigilate', 'https://www.emag.ro/placa-video-palit-geforce-gtx-1060-stormx-3g-3gb-gddr5-192-bit-ne51060015f9f/pd/DWZC1KBBM/#used-products', 'https://www.emag.ro/ecredit', 'https://www.emag.ro/resigilate', 'ht